<a href="https://colab.research.google.com/github/imran1386khan/Biobert-Training-Google-Colab/blob/master/1_train_bertffn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np

In [0]:
!pip install tensorflow==2.0.0-alpha0

In [2]:
from tensorflow.python.client import device_lib
!cat /proc/meminfo | egrep "MemTotal*"

device_lib.list_local_devices()

MemTotal:       13333556 kB


[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 15948324212355340709, name: "/device:XLA_CPU:0"
 device_type: "XLA_CPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 7424318645544884706
 physical_device_desc: "device: XLA_CPU device"]

In [0]:
import os
import urllib.request

#Link to pre-trained BERT
urllib.request.urlretrieve('https://github.com/naver/biobert-pretrained/releases/download/v1.0-pubmed-pmc/biobert_v1.0_pubmed_pmc.tar.gz', 'BioBert.tar.gz')

if not os.path.exists('BioBertFolder'):
    os.makedirs('BioBertFolder')
    
import tarfile
tar = tarfile.open("BioBert.tar.gz")
tar.extractall(path='BioBertFolder/')
tar.close()

!wget https://github.com/re-search/DocProduct/archive/v0.2.0_dev.zip
!wget https://github.com/re-search/gpt2-estimator/archive/master.zip

!unzip master.zip
!unzip v0.2.0_dev.zip

!mv DocProduct-0.2.0_dev/* /content/
!mv gpt2-estimator-master/* /content/

!wget  https://anaconda.org/pytorch/faiss-cpu/1.2.1/download/linux-64/faiss-cpu-1.2.1-py36_cuda9.0.176_1.tar.bz2

!tar xvjf faiss-cpu-1.2.1-py36_cuda9.0.176_1.tar.bz2
!cp -r lib/python3.6/site-packages/* /usr/local/lib/python3.6/dist-packages/
!pip install mkl

In [0]:
!mkdir '/content/data'
!cp '/content/drive/My Drive/data/QandA_Dataset.csv' '/content/data/train_data.csv'

In [0]:
# df = pd.read_csv('/content/data/train_data.csv')
# df = df.head(50)
# df.to_csv('/content/data/small.csv', index=False)
# !rm '/content/data/train_data.csv'

In [1]:
# import keras.backend as K
import numpy as np
import argparse
import cv2
import sys

import tensorflow as tf
# tf.compat.v1.enable_eager_execution()
tf.compat.v1.disable_eager_execution()
import tensorflow.keras.backend as K
from tensorflow.keras.models import load_model
import argparse
import os
import requests

from docproduct.dataset import create_dataset_for_bert
from docproduct.models import MedicalQAModelwithBert
from docproduct.loss import qa_pair_loss, qa_pair_cross_entropy_loss
from docproduct.tokenization import FullTokenizer
from docproduct.metrics import qa_pair_batch_accuracy

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or 

In [0]:
!mkdir '/content/drive/My Drive/DocAssistTraining'
!mkdir '/content/drive/My Drive/DocAssistTraining/bertffn_crossentropy'
!mkdir '/content/drive/My Drive/DocAssistTraining/bertffn_crossentropy/bertffn'

In [0]:
# This is used for Saving only one chkpoint and overriding it every epochs
checkpoint_path = '/content/models/bertffn.ckpt'
# checkpoint_path = "/content/drive/My Drive/DocAssistTraining/bertffn_crossentropy/bertffn/bertffn.ckpt"
# checkpoint_dir = os.path.dirname(checkpoint_path)

# Include the epoch in the file name (uses `str.format`). This can be used to save multiple chk points
# checkpoint_path = "/content/models/bertffn_crossentropy/bertffn/cp-{epoch:04d}.ckpt"
# checkpoint_dir = os.path.dirname(checkpoint_path)

model_path='/content/drive/My Drive/DocAssistTraining/bertffn_crossentropy/bertffn/final.ckpt'
data_path='data/'
num_epochs=2
num_gpu=1
batch_size=10
learning_rate=2e-5
validation_split=0.2
loss='categorical_crossentropy'
pretrained_path='/content/BioBertFolder/biobert_v1.0_pubmed_pmc/'
max_seq_len=512

K.set_floatx('float32')

if loss == 'categorical_crossentropy':
    loss_fn = qa_pair_cross_entropy_loss
else:
    loss_fn = qa_pair_loss

tokenizer = FullTokenizer(os.path.join(pretrained_path, 'vocab.txt'))

In [0]:
d = create_dataset_for_bert(
    data_path, tokenizer=tokenizer, batch_size=batch_size,
    shuffle_buffer=500000, dynamic_padding=True, max_seq_length=max_seq_len)

eval_d = create_dataset_for_bert(
    data_path, tokenizer=tokenizer, batch_size=batch_size,
    mode='eval', dynamic_padding=True, max_seq_length=max_seq_len,
    bucket_batch_sizes=[64, 64, 64])

In [16]:
for ele in d:
  print(ele)

({'a_input_ids': <tf.Tensor: id=231817, shape=(16, 88), dtype=int64, numpy=
array([[ 101, 1267, 1240, ...,    0,    0,    0],
       [ 101, 1930,  119, ...,    0,    0,    0],
       [ 101, 1321, 1123, ...,    0,    0,    0],
       ...,
       [ 101, 1242, 1535, ...,    0,    0,    0],
       [ 101, 1294, 1612, ...,    0,    0,    0],
       [ 101, 2812, 4455, ...,    0,    0,    0]])>, 'a_input_masks': <tf.Tensor: id=231818, shape=(16, 88), dtype=int64, numpy=
array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0]])>, 'a_segment_ids': <tf.Tensor: id=231819, shape=(16, 88), dtype=int64, numpy=
array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])>, 'q_input_ids': <tf.Tensor: id=231821, shape

In [4]:
medical_qa_model = MedicalQAModelwithBert(
    config_file=os.path.join(
        pretrained_path, 'bert_config.json'),
    checkpoint_file=os.path.join(pretrained_path, 'biobert_model.ckpt'))

optimizer = tf.keras.optimizers.Adam(lr=learning_rate)

medical_qa_model.compile(
    optimizer=optimizer, loss=loss_fn, metrics=[qa_pair_batch_accuracy])

epochs = num_epochs

callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, verbose=1, save_weights_only=True, save_best_only=False, period=1)

medical_qa_model.fit(d, epochs=epochs, callbacks=[callback])

medical_qa_model.summary()

# medical_qa_model.save_weights(model_path)

# Save the weights using the `checkpoint_path` format. Enable for saving multiple chk points
# medical_qa_model.save_weights(checkpoint_path.format(epoch=0))

medical_qa_model.evaluate(eval_d)


Epoch 1/2
[[[[0.230596691 -0.133663878 -0.262022525 ... 0.0568055846 -0.00428466499 0.012978144]
  [-0.281676114 0.76460129 -0.346016884 ... 0.0748428404 0.443163812 0.802191675]
  [-0.361676395 0.245992765 0.272511661 ... -0.248312011 0.467030674 -0.409435332]
  ...
  [-0.187291071 -0.925023 0.121136025 ... -0.512395501 -0.441329658 0.354023367]
  [-0.527089834 -1.22049916 0.0345268063 ... 0.170702472 0.23843044 0.157179028]
  [-0.341838181 -1.5209837 0.0613879375 ... -1.25804174 -0.587423205 0.211372718]]

 [[0.224181622 -0.139916822 -0.268119484 ... 0.0504326038 -0.0106474757 0.0067406483]
  [-0.283415556 0.778559208 -0.348138 ... 0.0846240073 0.451650858 0.0725840628]
  [0.0832337588 0.246459439 0.274271458 ... -0.255712599 0.470471442 -0.41930142]
  ...
  [-0.203127399 -0.966006577 0.116787255 ... -0.538770258 -0.467644274 0.357067645]
  [-0.519761086 -1.21602476 0.0445338376 ... 0.181260273 -0.566842377 0.53357321]
  [-0.240919635 -1.14638555 0.0611011349 ... -0.949317 -0.4175106

ValueError: ignored

# Junk

In [0]:
from docproduct.train_data_to_embedding import train_data_to_embedding

# tf.compat.v1.enable_eager_execution()

train_data_to_embedding(model_path='/content/drive/My Drive/DocAssistTraining/bertffn_crossentropy/bertffn/',
                            data_path='/content/data/',
                            output_path='/content/output.prkt',
                            pretrained_path='/content/BioBertFolder/biobert_v1.0_pubmed_pmc/')


0it [00:00, ?it/s]

Reading /content/data/small.csv


1it [01:53, 113.97s/it]


In [0]:
df = pd.read_parquet('/content/output.prkt')
df.head(20)

,question,answer,Q_FFNN_embeds,A_FFNN_embeds
0,Like 5 yrs ago to now I've had some reoccurrin...,test to see if you are a carrier of herpes sim...,"[1.0962960720062256, 0.35729750990867615, -0.5...","[2.1191976070404053, 0.20118729770183563, -0.1..."
1,My 6 year old daughter complains about vaginal...,Make sure she is not. Wearing tight fitting un...,"[0.8566192388534546, 0.3873278796672821, -0.41...","[2.0420145988464355, 0.045309700071811676, -0...."
2,My 6 year old daughter complains about vaginal...,Take her to a Gyn. Gynecologists are trained t...,"[0.8566192388534546, 0.3873278796672821, -0.41...","[2.107142448425293, 0.18660292029380798, -0.00..."
3,My Friend is experiencing irritation and she s...,Followup with doc. Some women use yoghurt as a...,"[0.9020317792892456, 0.5287120342254639, -0.52...","[1.675483226776123, 0.28767189383506775, -0.22..."
4,My Friend is experiencing irritation and she s...,Worrisome. Sounds like a self treatment for va...,"[0.9020317792892456, 0.5287120342254639, -0.52...","[1.5095821619033813, 0.22489629685878754, -0.3..."
5,"My VG is itchy and feels irritated, no pain, n...",Many women don't understand that yeast often c...,"[1.2272748947143555, 0.0677652359008789, -0.31...","[1.7241419553756714, 0.28745684027671814, -0.3..."
6,My anus has been itchy and burning. Went away ...,Red excoriated skin is likely from a dermatiti...,"[1.0458486080169678, 0.14292249083518982, -0.1...","[1.4487098455429077, -0.15864232182502747, -0...."
7,My anus itching and my pubic hair also my insi...,Infection. This sounds like 2 separate issues:...,"[0.9763892292976379, 0.38595932722091675, -0.3...","[2.031980037689209, 0.2574853003025055, -0.202..."
8,My anus itching and my pubic hair also my insi...,Burning. itching and burning can be from a yea...,"[0.9763892292976379, 0.38595932722091675, -0.3...","[2.5956954956054688, 0.32757827639579773, -0.3..."
9,My friend had redness in her vagina and and it...,Vaginitis? vulvitis? See her doctor.,"[0.9602717161178589, 0.24477048218250275, -0.1...","[2.3519704341888428, 0.11928334832191467, -0.1..."


In [0]:
tf.shape(df.Q_FFNN_embeds)

<tf.Tensor: id=24368, shape=(2,), dtype=int32, numpy=array([ 50, 768], dtype=int32)>

In [0]:
y_true = tf.eye(tf.shape(df.Q_FFNN_embeds)[0])

In [0]:
y_true

<tf.Tensor: id=24421, shape=(50, 50), dtype=float32, numpy=
array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.]], dtype=float32)>

In [0]:
q_embedding = df.Q_FFNN_embeds
a_embedding = df.A_FFNN_embeds
similarity_matrix = tf.matmul(
        a=q_embedding, b=a_embedding, transpose_b=True)

In [0]:
similarity_matrix

<tf.Tensor: id=24372, shape=(50, 50), dtype=float64, numpy=
array([[667.7659385 , 693.15932498, 653.16278176, ..., 676.82797045,
        663.0307033 , 711.21231976],
       [656.57736511, 689.9612447 , 652.60367404, ..., 674.5995719 ,
        658.23500307, 705.80929928],
       [656.57736511, 689.9612447 , 652.60367404, ..., 674.5995719 ,
        658.23500307, 705.80929928],
       ...,
       [690.9488539 , 722.29331749, 677.60918259, ..., 701.22186118,
        687.42525986, 745.86759126],
       [663.90166992, 691.34643131, 650.98480913, ..., 676.07821412,
        659.47771479, 704.0814083 ],
       [663.90172161, 691.34648363, 650.98485881, ..., 676.078267  ,
        659.47776484, 704.08146074]])>

In [0]:
similarity_matrix_softmaxed = tf.nn.softmax(similarity_matrix)

In [0]:
similarity_matrix_softmaxed

<tf.Tensor: id=24374, shape=(50, 50), dtype=float64, numpy=
array([[1.84702354e-30, 1.97097018e-19, 8.40237459e-37, ...,
        1.59243934e-26, 1.62175748e-32, 1.36457079e-11],
       [4.73812145e-32, 1.49293493e-17, 8.90951539e-34, ...,
        3.18090827e-24, 2.48604625e-31, 1.13962910e-10],
       [4.73812145e-32, 1.49293493e-17, 8.90951539e-34, ...,
        3.18090827e-24, 2.48604625e-31, 1.13962910e-10],
       ...,
       [5.68854713e-34, 2.33200211e-20, 9.15494464e-40, ...,
        1.64631045e-29, 1.67773712e-35, 4.03557991e-10],
       [2.30349482e-29, 1.91203425e-17, 5.65803819e-35, ...,
        4.47294125e-24, 2.76113689e-31, 6.48971909e-12],
       [2.30349233e-29, 1.91203341e-17, 5.65802074e-35, ...,
        4.47294177e-24, 2.76112939e-31, 6.48971706e-12]])>

In [0]:
y_true = tf.argmax(y_true, axis=1)
y_pred = tf.argmax(similarity_matrix, axis=1)
print(y_true)
print(y_pred)

tf.Tensor(
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49], shape=(50,), dtype=int64)
tf.Tensor(
[18 45 45 45 45 18 18 18 18 45 45 45 18 45 45 18 18 45 18 18 18 45 45 45
 45 45 45 45 45 18 18 45 45 45 45 45 18 18 18 18 18 18 18 18 18 18 18 18
 45 45], shape=(50,), dtype=int64)


In [0]:
tf.cast(tf.equal(y_pred, y_true), tf.float32)

<tf.Tensor: id=24438, shape=(50,), dtype=float32, numpy=
array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
      dtype=float32)>

In [0]:
acc = tf.reduce_mean(tf.cast(tf.equal(y_pred, y_true), tf.float32))

In [0]:
acc

<tf.Tensor: id=24432, shape=(), dtype=float32, numpy=0.02>

In [0]:
tf.argmax(similarity_matrix_softmaxed[0])

<tf.Tensor: id=24381, shape=(), dtype=int64, numpy=18>

In [0]:
loss = tf.keras.losses.categorical_crossentropy(y_true, similarity_matrix_softmaxed, from_logits=False)

In [0]:
loss

<tf.Tensor: id=24719, shape=(50,), dtype=float64, numpy=
array([16.11809565, 16.11809565, 16.11809565, 16.11809565, 16.11809565,
       16.11809565, 16.11809565, 16.11809565, 16.11809565, 16.11809565,
       16.11809565, 16.11809565, 16.11809565, 15.07356181, 16.11809565,
       16.11809565, 16.11809565, 16.11809565,  2.95342164, 16.11809565,
       16.11809565, 16.11809565, 16.11809565, 16.11809565, 16.11809565,
       16.11809565, 16.11809565, 16.11809565, 16.11809565, 16.11809565,
       16.11809565, 16.11809565, 16.11809565, 16.11809565, 16.11809565,
       16.11809565, 16.11809565, 16.11809565, 16.11809565, 16.11809565,
       16.11809565, 16.11809565, 16.11809565, 16.11809565, 16.11809565,
        2.06206044, 16.11809565, 16.11809565, 16.11809565, 16.11809565])>

In [0]:
labels_cce  = K.variable([[1,    0,  0], [0,    1,  0], [0,   0,   1]])
preds       = K.variable([[.90,.05,.05], [.50,.89,.60], [.05,.01,.94]])

In [0]:
losses = []

np_labels = K.get_value(labels_cce)
np_preds  = K.get_value(preds)

for label, pred in zip(np_labels, np_preds):
    pred /= pred.sum(axis=-1, keepdims=True)
    print('pred : ',pred)
    print('label : ',label)
    losses.append(np.sum(label * -np.log(pred), axis=-1, keepdims=False))
print(losses)

pred :  [0.9  0.05 0.05]
label :  [1. 0. 0.]
pred :  [0.2512563  0.44723618 0.30150756]
label :  [0. 1. 0.]
pred :  [0.05 0.01 0.94]
label :  [0. 0. 1.]
[0.105360545, 0.8046685, 0.061875407]


In [0]:
pred = [3.49840608e-29 , 0.05, 0.05]
-np.log(pred) * [1,0,0]


array([65.52266024,  0.        ,  0.        ])

In [0]:
import numpy as np

In [0]:
x = [3.49840608e-29, 5.21724993e-17, 1.32909681e-37, 8.69946029e-24,
       6.42280146e-19, 1.06560527e-22, 3.20415741e-29, 3.07536680e-32,
       2.19788681e-13, 1.47057614e-25, 2.37063794e-23, 4.86786939e-22,
       6.53078132e-32, 1.19627823e-04, 4.75464243e-22, 1.11019827e-20,
       1.30980806e-31, 3.30851996e-24, 5.13415064e-01, 5.57493394e-28,
       4.55147633e-12, 1.01944511e-24, 4.30757341e-12, 2.23792386e-20,
       2.15366987e-19, 1.18201880e-19, 1.06912472e-19, 6.11925136e-19,
       1.21602987e-24, 1.86776670e-58, 3.30901644e-14, 1.43814301e-25,
       7.44197436e-22, 3.90050859e-26, 2.75638425e-28, 1.26396912e-49,
       1.09015236e-12, 3.65601432e-19, 8.72899811e-16, 5.22431074e-37,
       1.68621718e-41, 1.16407291e-23, 1.29391908e-13, 6.41339662e-31,
       2.43685044e-29, 4.86465308e-01, 5.52473964e-29, 1.42386132e-26,
       4.90662621e-31, 6.65340795e-12]

In [0]:
np.sum(x)

0.9999999998399859

In [0]:
 x /= sum(np.array(x))

In [0]:
x.sum()

1.0

In [0]:
y = [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]

In [0]:
-np.log(x) * y

array([65.52266024,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ])

In [19]:
t = tf.constant([
  [[0, 1, 2, 3, 4],
   [5, 6, 7, 8, 9]],
  [[10, 11, 12, 13, 14],
   [15, 16, 17, 18, 19]],
  [[20, 21, 22, 23, 24],
   [25, 26, 27, 28, 29]]])
print(t)
print(t[-2])

# tf.reshape(t, [-1])

tf.Tensor(
[[[ 0  1  2  3  4]
  [ 5  6  7  8  9]]

 [[10 11 12 13 14]
  [15 16 17 18 19]]

 [[20 21 22 23 24]
  [25 26 27 28 29]]], shape=(3, 2, 5), dtype=int32)
tf.Tensor(
[[10 11 12 13 14]
 [15 16 17 18 19]], shape=(2, 5), dtype=int32)


In [5]:
len([25, 26, 27, 28, 29])

5